In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
import os

In [ ]:
## Reading from a pdf file
from langchain_community.document_loaders import TextLoader

loader=TextLoader('Ashtangahridaya.txt')
docs=loader.load()
# 3. Split into smaller chunks (important for embeddings & retrieval)
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,chunk_overlap=50)
Final_document = text_splitter.split_documents(docs)

In [ ]:
###Reading a pdf file

from langchain_community.document_loaders import PyPDFLoader

loader=PyPDFLoader("/content/Charaka-Samhita-Acharya-Charaka.pdf")
docs=loader.load()

# 3. Split into smaller chunks (important for embeddings & retrieval)
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,chunk_overlap=50)
Final_document = text_splitter.split_documents(docs)

# apply preprocessing for both the files and create a clean document

In [ ]:
import re

def preprocess_text_mupdf(text):
    # Remove headers/footers
    text = re.sub(r'\n\s*\n', '\n', text)  # Remove empty lines
    text = re.sub(r'[^A-Za-z0-9.,;:!?()\'\"\n]+', ' ', text)  # Remove special characters but keep punctuation
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
    return text.strip()

for i in range(len(Final_document)):
  Final_document[i].page_content=preprocess_text_mupdf(Final_document[i].page_content)

# Save the cleaned text to a file
with open('Ashtang_cleaned.txt', 'w') as file:
    for document in Final_document:
        cleaned_text_mupdf = document.page_content
        # Write the cleaned text to the file
        file.write(cleaned_text_mupdf)


In [ ]:


# 1. Load multiple books
loader1 = TextLoader("D:\\AIBootcamp\\AyuMitra\\bookbank\\Ashtang_cleaned.txt", encoding="utf-8")
loader2 = TextLoader("D:\\AIBootcamp\\AyuMitra\\bookbank\\Charak_cleaned.txt", encoding="utf-8")

docs1 = loader1.load()
docs2 = loader2.load()

# 2. Combine documents
all_docs = docs1 + docs2

# 3. Split into smaller chunks (important for embeddings & retrieval)
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,chunk_overlap=50)
Final_document = text_splitter.split_documents(all_docs)


In [3]:
len(Final_document)  # Check the number of chunks created

4524

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\Asus\AppData\Local\Temp\ipykernel_17788\4286477000.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
c:\Users\Asus\anaconda3\envs\vedbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
"""
not needed
from langchain.vectorstores import Chroma

db = Chroma.from_documents(Final_document, embedding=embeddings, collection_name="bookstore")
"""


In [ ]:
# Create a persistent Chroma DB
persist_directory = r"D:\AIBootcamp\AyuMitra\db"  # use raw string for Windows paths

db = Chroma.from_documents(
    documents=Final_document,
    embedding=embeddings,
    collection_name="bookstore",
    persist_directory=persist_directory
)
db.persist()
print(" Chroma DB saved at", persist_directory)


✅ Chroma DB saved at D:\AIBootcamp\AyuMitra\db


C:\Users\Asus\AppData\Local\Temp\ipykernel_17788\2623306549.py:10: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [11]:
from langchain_core.runnables import RunnableMap
#To get a callable obebject to map result
retriever=db.as_retriever()
# This gets relevant context documents and prepares inputs for the prompt
retrieval_chain = (
    RunnableMap({
        "question": lambda x: x["question"],
        "context": lambda x: "\n\n".join(
            doc.page_content for doc in retriever.get_relevant_documents(x["question"])
        )
    })
)



In [24]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate

# Load environment variables from .env file
load_dotenv()

# Fetch system prompt path from env
system_prompt_path = os.getenv("SYSTEM_PROMPT_PATH")
print(system_prompt_path)

if not system_prompt_path:
    raise ValueError("SYSTEM_PROMPT_PATH not set in .env file")

# Load system prompt from text file
with open(system_prompt_path, "r", encoding="utf-8") as f:
    system_prompt = f.read()

# Build Ayurvedic Medical Bot Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        (
            "user",
            "Context:\n{context}\n\nQuestion: {question}"
        ),
    ]
)


D:\AIBootcamp\Ayurvedic_chatbot\system_prompt.txt


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GEMINI_API_KEY,
    temperature=0.5,
    max_output_tokens=4096
)




In [28]:
from langchain_core.output_parsers import StrOutputParser

# For parsing output
output_parser=StrOutputParser()

# Final chain
chain = retrieval_chain | prompt | llm | output_parser



In [29]:
query = " I feel severe pain in urinary bladder,intense pain in penis ,frequent urination in less quantity"


rag_answer=chain.invoke({"question":query})
print(rag_answer)

C:\Users\Asus\AppData\Local\Temp\ipykernel_21712\49847817.py:9: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  doc.page_content for doc in retriever.get_relevant_documents(x["question"])


Okay, based on the context you've provided, here's a possible Ayurvedic approach to address your symptoms:

**Understanding the Imbalance:**

*   The symptoms suggest an imbalance of Vata dosha, particularly in the Apana Vata subtype, which governs the downward flow of energy and elimination.
*   The pain and urinary issues indicate a possible obstruction (Ama) in the urinary channels (Mutravaha Srotas).
*   The mention of suppressed urges and decreased malas (waste products) further points to Vata aggravation.

**Treatment Approach:**

The general line of treatment involves Vata pacification, Ama digestion, and restoring healthy flow in the urinary system.

**1. Home Remedy (Vata-Pacifying & Diuretic):**

*   **Ingredients:**
    *   Warm water
    *   1/2 teaspoon of ginger powder
    *   1/4 teaspoon of cumin powder
*   **Preparation:** Mix the ginger and cumin powder in warm water.
*   **Dosage:** Drink this twice a day, preferably after meals.
*   **Timing:** Continue for a week a

In [30]:
import numpy as np
from scipy.spatial.distance import cosine
def get_embedding(text):
    """
    Generate an embedding for a given text.

    Args:
    - text (str): The input text.

    Returns:
    - The sentence embedding.
    """
    # Generate the sentence embeddings
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    return embeddings.embed_query(text)


def calculate_cosine_similarity(embedding1, embedding2):
    """
    Calculate the cosine similarity between two embeddings.

    Args:
    - embedding1 (torch.Tensor): The first embedding.
    - embedding2 (torch.Tensor): The second embedding.

    Returns:
    - The cosine similarity score.
    """
    # Calculate the cosine similarity
    similarity = 1 - cosine(embedding1, embedding2)
    return similarity


def calculate_similarity_scores(true_answer, rag_answer, non_rag_answer):
    """
    Calculate the cosine similarity scores between the true answer and both RAG-based and non-RAG-based answers.

    Args:
    - true_answer (str): The true answer text.
    - rag_answer (str): The RAG-based model's answer text.
    - non_rag_answer (str): The non-RAG-based model's answer text.

    Returns:
    - A dictionary with cosine similarity scores.
    """
    # Convert the answers to embeddings
    true_answer_embedding = get_embedding(true_answer)
    rag_answer_embedding = get_embedding(rag_answer)
    non_rag_answer_embedding = get_embedding(non_rag_answer)

    # Calculate cosine similarity scores
    rag_similarity = calculate_cosine_similarity(true_answer_embedding, rag_answer_embedding)
    non_rag_similarity = calculate_cosine_similarity(true_answer_embedding, non_rag_answer_embedding)


    # Return the scores
    return {
        "RAG Similarity Score": rag_similarity,
        "Non-RAG Similarity Score": non_rag_similarity
    }




In [31]:
true_answer = """**Ayurvedic Remedies**

*   **Classical Formulation:**
    *   **Punarnavadi Kashayam:** This classical Ayurvedic decoction is known for its diuretic and anti-inflammatory properties, which can help alleviate bladder pain and promote healthy urine flow.
        *   **Ingredients:** Punarnava (Boerhavia diffusa) and other herbs.
        *   **Preparation:** Usually available as a ready-made decoction.
        *   **Dosage:** 15-20 ml, mixed with an equal amount of warm water, twice daily after food.
        *   **Timing:** Morning and evening after meals.

**Lifestyle Adjustments**

*   **Hydration:** Drink plenty of fluids (warm water, herbal teas) to help flush out the urinary system and reduce burning sensations.
*   **Avoid Suppressing Urges:** Always heed the natural urge to urinate. Suppressing it can lead to imbalances and pain.
*   **Warm Compress:** Apply a warm compress to the lower abdomen to help soothe the bladder and alleviate pain.

**Precautions:**

*   If you have a known history of kidney stones or any chronic urinary condition, consult with a healthcare provider before trying these remedies.
*   If you experience fever, severe pain, or blood in the urine, seek immediate medical attention.
"""

In [32]:
# Calculate the similarity scores
similarity_scores = calculate_similarity_scores(true_answer, rag_answer, true_answer)
print(similarity_scores)

{'RAG Similarity Score': np.float64(0.7142274190583674), 'Non-RAG Similarity Score': np.float64(1.0)}


In [ ]:
from rag_logic import load_db, build_chain,build_conversational_chain,get_session_history
from dotenv import load_dotenv
import os

load_dotenv()
db=load_db(embeddings)  # or "Cases" based on your choice
chain, retriever, config=build_conversational_chain(db)
from langchain_core.messages import HumanMessage
query = " I feel severe pain in urinary bladder,intense pain in penis ,frequent urination in less quantity"

rag_answer=chain.invoke(
    [HumanMessage(content=query)
     ],
     config=config
)



print(rag_answer)

True

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db=load_db(embeddings)  # or "Cases" based on your choice

C:\Users\Asus\AppData\Local\Temp\ipykernel_17596\2766215191.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
c:\Users\Asus\anaconda3\envs\vedbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
chain,retriever=build_chain(db)

In [5]:
query = " I feel severe pain in urinary bladder,intense pain in penis ,frequent urination in less quantity"


rag_answer=chain.invoke({"question":query})
print(rag_answer)

d:\AIBootcamp\Ayurvedic_chatbot\src\rag_logic.py:71: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  doc.page_content for doc in retriever.get_relevant_documents(x["question"])


Based on your symptoms, it appears you may be experiencing a condition related to urinary function and pain. In Ayurveda, this could be related to an imbalance of Vata and Pitta doshas affecting the Apana Vata (the Vata subtype governing the lower abdomen) and potentially causing inflammation or obstruction in the urinary tract.

Here are a few potential remedies and lifestyle adjustments that may help alleviate your symptoms:

**Home Remedy:**
*   **Ingredients:**
    *   1 teaspoon of coriander seeds
    *   1 teaspoon of cumin seeds
    *   2 cups of water
*   **Preparation:**
    *   Boil the water with the coriander and cumin seeds.
    *   Simmer until the water reduces to half.
    *   Strain and let it cool slightly.
*   **Dosage:** Drink this twice a day.
*   **Timing:** Morning and evening.

**Classical Ayurvedic Formulation:**
*   **Chandrashuradi Guti:** This tablet is known for its diuretic and anti-inflammatory properties, which can help in urinary disorders.
    *   **Do

In [1]:
from rag_logic import load_db, build_conversational_chain
from langchain_community.embeddings import HuggingFaceEmbeddings
from dotenv import load_dotenv

load_dotenv()

# Load embeddings + DB
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = load_db(embeddings)

# Get conversational chain
chain, retriever, config = build_conversational_chain(db)

# Query
query = "I feel severe pain in urinary bladder, intense pain in penis, frequent urination in less quantity"

# ✅ Pass dict, not list
rag_answer = chain.invoke({"question": query}, config=config)
print(rag_answer)



C:\Users\Asus\AppData\Local\Temp\ipykernel_8592\686652566.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\Asus\anaconda3\envs\vedbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\AIBootcamp\Ayurvedic_chatbot\src\rag_logic.py:62: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in

Based on your symptoms (severe pain in the urinary bladder and penis, frequent urination in small quantities), here are a few potential Ayurvedic remedies:

*   **Home Remedy:**
    *   **Ingredients:** Gokshura (Tribulus terrestris) powder, warm water.
    *   **Preparation:** Mix 1 teaspoon of Gokshura powder in a glass of warm water.
    *   **Dosage:** Drink this twice daily.
    *   **Timing:** Morning and evening, preferably before meals.
    *   **Benefits:** Gokshura is known for its diuretic and anti-inflammatory properties, which can help alleviate urinary pain and promote healthy urine flow.

*   **Classical Ayurvedic Formulation:**
    *   **Medication:** Chandraprabha Vati
    *   **Dosage:** 1-2 tablets twice daily
    *   **Timing:** After food with warm water
    *   **Benefits:** It helps in relieving urinary problems.

*   **Lifestyle Adjustment:**
    *   **Hydration:** Drink plenty of water throughout the day to help flush out toxins and reduce irritation in the uri

In [2]:
query = "give me only classical formulations from above answer with no changes"

# ✅ Pass dict, not list
rag_answer = chain.invoke({"question": query}, config=config)
print(rag_answer)

*   **Classical Ayurvedic Formulation:**
    *   **Medication:** Chandraprabha Vati
    *   **Dosage:** 1-2 tablets twice daily
    *   **Timing:** After food with warm water
    *   **Benefits:** It helps in relieving urinary problems.

Please confirm this remedy or medication with a certified Ayurvedic practitioner or healthcare expert before following it.


In [2]:
from rag_logic import save_feedback
query = "give me only classical formulations from above answer with no changes"

rag_answer="testing"
save_feedback(
    query=query,
    bot_output=rag_answer,
    doctor_feedback="correct"
)

In [8]:
# Query
query = "I have legs stiffness+thighs stiffness+wrist stiffness"

# ✅ Pass dict, not list
rag_answer = chain.invoke({"question": query}, config=config)

In [11]:
import pandas as pd
from rag_logic import load_db, build_conversational_chain
from langchain_huggingface import HuggingFaceEmbeddings

# ================================
# Load Embeddings + DB + Chain
# ================================
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = load_db(embeddings)
conv_chain, retriever, config = build_conversational_chain(db, session_id="excel_run")

# ================================
# File paths
# ================================
input_file = "Filtered.xlsx"
output_file = "Disease_symptoms_with_answers.xlsx"

# Load all sheets
sheets_dict = pd.read_excel(input_file)

output_sheets = {}

In [26]:
results = []
for query in sheets_dict['symptoms']:  # assuming column is "symptom"
    rag_answer = conv_chain.invoke({"question": query}, config=config)
    results.append({"query": query, "RAG_Answer": rag_answer})

# Convert to DataFrame for saving
output_df = pd.DataFrame(results)

In [27]:
output_df.head()

,query,RAG_Answer
0,fever+cough+hiccups+asthama+change in taste of...,"Based on the symptoms provided, here is an Ayu..."
1,tachycardia+feeling of dryness in heart++stiff...,"Based on the symptoms provided, here is an Ayu..."
2,heart burn+syncope+to be afraid+fever+heat up+...,"Based on the symptoms provided, here is an Ayu..."
3,heaviness in heart+stiffness in heart+dribblin...,"Based on the symptoms provided, here is an Ayu..."
4,intense heart ache+pricking pain in heart+itch...,"Based on the symptoms provided, here is an Ayu..."


In [28]:
output_df.to_excel(output_file, index=False)


In [30]:
from langchain_google_genai import ChatGoogleGenerativeAI
# Use the same model name as in conv_chain config
from dotenv import load_dotenv
load_dotenv()
import os
os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")
gemini_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")


In [31]:
import pandas as pd
import time
from rag_logic import load_db, build_conversational_chain
from sklearn.metrics.pairwise import cosine_similarity
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings

# ================================
# Load Gemini LLM (baseline)
# ================================
gemini_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# ================================
# Embeddings for similarity
# ================================
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# ================================
# Collect results
# ================================
baseline_results = []

for query in sheets_dict['symptoms']:
    # Direct Gemini answer
    gemini_answer = gemini_llm.invoke(query)
    gemini_text = gemini_answer.content if hasattr(gemini_answer, "content") else str(gemini_answer)

    # RAG answer
    rag_answer = conv_chain.invoke({"question": query}, config=config)
    rag_text = rag_answer if isinstance(rag_answer, str) else str(rag_answer)

    # Compute embeddings
    gemini_vec = embeddings_model.embed_query(gemini_text)
    rag_vec = embeddings_model.embed_query(rag_text)

    # Cosine similarity
    similarity = cosine_similarity([gemini_vec], [rag_vec])[0][0]

    # Save results
    baseline_results.append({
        "query": query,
        "Gemini_Answer": gemini_text,
        "RAG_Answer": rag_text,
        "Similarity_Score": round(similarity, 4)
    })

    time.sleep(5)  # avoid hitting Gemini quota

# ================================
# Save to Excel
# ================================
baseline_df = pd.DataFrame(baseline_results)
baseline_df.to_excel("RAG_vs_Gemini_with_Similarity.xlsx", index=False)

print("✅ Saved RAG vs Gemini results with similarity to RAG_vs_Gemini_with_Similarity.xlsx")


✅ Saved RAG vs Gemini results with similarity to RAG_vs_Gemini_with_Similarity.xlsx
